In [17]:
import asyncio
import websockets
import json
import time
import pandas as pd
import jiwer
from tqdm.asyncio import tqdm

WS_URL = "ws://127.0.0.1:8000/api/v1/ws"

In [18]:
# --- 평가 지표 함수 ---
def normalize(text):
    transformation = jiwer.Compose([
        jiwer.ToLowerCase(),
        jiwer.RemovePunctuation(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
    ])
    return transformation(text)

def calculate_metrics(truth, hypothesis):
    norm_truth = normalize(truth)
    norm_hyp = normalize(hypothesis)
    if not norm_truth: return 0.0, 0.0
    
    wer = jiwer.wer(norm_truth, norm_hyp)
    cer = jiwer.cer(norm_truth, norm_hyp)
    return wer, cer

In [19]:
import os
import glob

BASE_DIR = r"C:\backend\tests\test_dataset"
WAV_ROOT = os.path.join(BASE_DIR, "wav")
LABEL_ROOT = os.path.join(BASE_DIR, "label")

def load_dataset_separated(wav_root, label_root):
    dataset = []
    
    search_pattern = os.path.join(wav_root, "**", "*.wav")
    wav_files = glob.glob(search_pattern, recursive=True)
    
    print(f"검색된 WAV 파일 수: {len(wav_files)}")

    for wav_path in wav_files:
        rel_path = os.path.relpath(wav_path, wav_root)
        
        rel_txt_path = rel_path.replace(".wav", ".txt").replace(".WAV", ".txt")
        target_txt_path = os.path.join(label_root, rel_txt_path)
        
        if os.path.exists(target_txt_path):
            try:
                with open(target_txt_path, 'r', encoding='utf-8') as f:
                    ground_truth = f.read().strip()
            except UnicodeDecodeError:
                with open(target_txt_path, 'r', encoding='cp949') as f:
                    ground_truth = f.read().strip()

            folder_name = os.path.dirname(rel_path) # S00000365
            file_name = os.path.basename(wav_path)  # 0001.wav
            
            unique_id = f"{folder_name}_{file_name}"
            
            dataset.append({
                "id": unique_id,           # 고유 식별자
                "folder": folder_name,     # 폴더 정보
                "audio_path": wav_path,    # 실제 경로
                "truth": ground_truth      # 정답
            })
        else:
            pass

    return dataset

# 실행
data = load_dataset_separated(WAV_ROOT, LABEL_ROOT)

검색된 WAV 파일 수: 321


In [20]:
print(data)

[{'id': 'S00000365_0001.wav', 'folder': 'S00000365', 'audio_path': 'C:\\backend\\tests\\test_dataset\\wav\\S00000365\\0001.wav', 'truth': '네 고객님 중앙교육의 이진혁 상담원입니다.'}, {'id': 'S00000365_0002.wav', 'folder': 'S00000365', 'audio_path': 'C:\\backend\\tests\\test_dataset\\wav\\S00000365\\0002.wav', 'truth': '네 안녕하세요. 결제 때문에 전화 드렸어요.'}, {'id': 'S00000365_0003.wav', 'folder': 'S00000365', 'audio_path': 'C:\\backend\\tests\\test_dataset\\wav\\S00000365\\0003.wav', 'truth': 'n/ 어/ 분당 살 때'}, {'id': 'S00000365_0004.wav', 'folder': 'S00000365', 'audio_path': 'C:\\backend\\tests\\test_dataset\\wav\\S00000365\\0004.wav', 'truth': '중앙교육 학습지 몇 년 했었고요.'}, {'id': 'S00000365_0005.wav', 'folder': 'S00000365', 'audio_path': 'C:\\backend\\tests\\test_dataset\\wav\\S00000365\\0005.wav', 'truth': '지금은 학습지 이용 안 하고 있거든요. 그리고 저희 서초로 이사도 했고요.'}, {'id': 'S00000365_0006.wav', 'folder': 'S00000365', 'audio_path': 'C:\\backend\\tests\\test_dataset\\wav\\S00000365\\0006.wav', 'truth': '근데 며칠 전에 갑자기 여기 학습지 결제 문자가 뜨더라고요.

In [21]:
import asyncio
import websockets
import json
import time
import wave
import contextlib
import pandas as pd
from tqdm.asyncio import tqdm

async def process_single_item(websocket, file_item):
    audio_path = file_item['audio_path'] 
    ground_truth = file_item['truth']
    
    # 오디오 파일 읽기
    with open(audio_path, "rb") as f:
        audio_bytes = f.read()
        
    # 오디오 길이 계산
    with contextlib.closing(wave.open(audio_path, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        audio_duration = frames / float(rate)

    result_data = {
        "file": file_item['id'],
        "duration": audio_duration,
        "latency": None,
        "rtf": None,              
        "wer": None,
        "cer": None,
        "truth": ground_truth,
        "hypothesis": "",
        "status": "fail",
        "error_msg": ""
    }

    try:
        start_time = time.perf_counter()
        
        await websocket.send(audio_bytes)
        
        # 응답 대기 (30초 타임아웃)
        response = await asyncio.wait_for(websocket.recv(), timeout=30.0)
        end_time = time.perf_counter()
        
        # 결과 파싱
        resp_json = json.loads(response)
        
        if isinstance(resp_json, dict):
            hypothesis = resp_json.get("text", "")
        else:
            hypothesis = str(resp_json)
        
        # 지표 계산
        wer, cer = calculate_metrics(ground_truth, hypothesis)
        latency = end_time - start_time
        rtf = latency / audio_duration if audio_duration > 0 else 0
        
        result_data.update({
            "latency": latency,
            "rtf": rtf,
            "wer": wer,
            "cer": cer,
            "hypothesis": hypothesis,
            "status": "success"
        })
        
    except Exception as e:
        print(f"에러 ({file_item['id']}): {e}")
        result_data["error_msg"] = str(e)
        
    return result_data

In [22]:
# --- 배치 실행 함수 ---
async def run_batch_test(dataset):
    results = []
    
    try:
        async with websockets.connect(WS_URL, ping_interval=None) as websocket:
            print("연결 성공")
            
            # tqdm으로 진행 상황 표시
            for item in tqdm(dataset):
                # 연결된 websocket 객체를 인자로 넘김
                res = await process_single_item(websocket, item)
                results.append(res)
                
    except Exception as e:
        print(f"에러 : {e}")
        
    return results

In [23]:
# --- 실행 및 분석 ---
results = await run_batch_test(data)

연결 성공


100%|██████████| 321/321 [06:38<00:00,  1.24s/it]


In [24]:
print(results)

[{'file': 'S00000365_0001.wav', 'duration': 4.10725, 'latency': 1.0202085000055376, 'rtf': 0.2483921115114828, 'wer': 0.0, 'cer': 0.0, 'truth': '네 고객님 중앙교육의 이진혁 상담원입니다.', 'hypothesis': '네, 고객님. 중앙교육의 이진혁 상담원입니다.', 'status': 'success', 'error_msg': ''}, {'file': 'S00000365_0002.wav', 'duration': 3.157375, 'latency': 1.439488399977563, 'rtf': 0.45591302901225317, 'wer': 0.5, 'cer': 0.09090909090909091, 'truth': '네 안녕하세요. 결제 때문에 전화 드렸어요.', 'hypothesis': '예, 안녕하세요. 결제 때문에 전화드렸어요.', 'status': 'success', 'error_msg': ''}, {'file': 'S00000365_0003.wav', 'duration': 1.690375, 'latency': 1.4824691000103485, 'rtf': 0.8770060489597565, 'wer': 0.4, 'cer': 0.4, 'truth': 'n/ 어/ 분당 살 때', 'hypothesis': '분당 살 때', 'status': 'success', 'error_msg': ''}, {'file': 'S00000365_0004.wav', 'duration': 2.7, 'latency': 0.9068489000201225, 'rtf': 0.3358699629704157, 'wer': 0.4, 'cer': 0.058823529411764705, 'truth': '중앙교육 학습지 몇 년 했었고요.', 'hypothesis': '중앙교육학습지 몇 년 했었고요.', 'status': 'success', 'error_msg': ''}, {'f

In [25]:
# 결과 분석
if results:
    df = pd.DataFrame(results)
    success_df = df[df['status'] == 'success']

    print("\n=== STT 성능 평가 결과 ===")
    print(f"총 시도: {len(df)} | 성공: {len(success_df)} | 실패: {len(df) - len(success_df)}")

    if not success_df.empty:
        print("-" * 40)
        print(f"1. 평균 변환 시간 : {success_df['latency'].mean():.4f} 초")
        print(f"2. p95 변환 시간  : {success_df['latency'].quantile(0.95):.4f} 초")
        print(f"3. RTF (평균)     : {success_df['rtf'].mean():.4f}")
        print(f"4. WER (단어 오류): {success_df['wer'].mean():.4f}")
        print(f"5. CER (음절 오류): {success_df['cer'].mean():.4f}")
        print("-" * 40)


=== STT 성능 평가 결과 ===
총 시도: 321 | 성공: 321 | 실패: 0
----------------------------------------
1. 평균 변환 시간 : 1.2313 초
2. p95 변환 시간  : 2.3645 초
3. RTF (평균)     : 0.3964
4. WER (단어 오류): 0.2576
5. CER (음절 오류): 0.1167
----------------------------------------
